In [1]:
from Bio import Align
from Bio.Align import substitution_matrices
import numpy as np



In [2]:
def needleman_wunsch_global_alignment(seq1, seq2, match_score=1, mismatch_score=-1, gap_penalty=-2):
    """
    Custom implementation of Needleman-Wunsch algorithm for global alignment
    """
    
    m, n = len(seq1), len(seq2)
    score_matrix = np.zeros((m + 1, n + 1))
    traceback_matrix = np.zeros((m + 1, n + 1))  
    
    
    for i in range(1, m + 1):
        score_matrix[i][0] = i * gap_penalty
        traceback_matrix[i][0] = 2  
    for j in range(1, n + 1):
        score_matrix[0][j] = j * gap_penalty
        traceback_matrix[0][j] = 3  
    
    
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            
            if seq1[i-1] == seq2[j-1]:
                diagonal_score = score_matrix[i-1][j-1] + match_score
            else:
                diagonal_score = score_matrix[i-1][j-1] + mismatch_score
            
            
            up_score = score_matrix[i-1][j] + gap_penalty
            left_score = score_matrix[i][j-1] + gap_penalty
            
            
            scores = [diagonal_score, up_score, left_score]
            max_score = max(scores)
            score_matrix[i][j] = max_score
            
            
            traceback_matrix[i][j] = scores.index(max_score) + 1
    
    
    aligned_seq1 = []
    aligned_seq2 = []
    i, j = m, n
    
    while i > 0 or j > 0:
        if traceback_matrix[i][j] == 1:  
            aligned_seq1.append(seq1[i-1])
            aligned_seq2.append(seq2[j-1])
            i -= 1
            j -= 1
        elif traceback_matrix[i][j] == 2:  
            aligned_seq1.append(seq1[i-1])
            aligned_seq2.append('-')
            i -= 1
        else:  
            aligned_seq1.append('-')
            aligned_seq2.append(seq2[j-1])
            j -= 1
    
    
    aligned_seq1 = ''.join(reversed(aligned_seq1))
    aligned_seq2 = ''.join(reversed(aligned_seq2))
    
    return score_matrix[m][n], aligned_seq1, aligned_seq2, score_matrix



In [ ]:

def align_with_biopython(seq1, seq2):
    """
    Use Biopython's PairwiseAligner for Smith-Waterman alignment
    """
    
    aligner = Align.PairwiseAligner()
    
    
    aligner.mode = 'local'
    
    
    aligner.match_score = 2.0
    aligner.mismatch_score = -1.0
    aligner.open_gap_score = -1.0
    aligner.extend_gap_score = -1.0
    
    
    alignments = aligner.align(seq1, seq2)
    
    
    best_alignment = next(alignments)
    
    return best_alignment


if __name__ == "__main__":
    
    sequence_X = "MAGNIFICENT"
    sequence_Y = "MAGNIFIC"
    
    
    print("SMITH-WATERMAN LOCAL ALIGNMENT")
    
    print(f"Sequence X: {sequence_X}")
    print(f"Sequence Y: {sequence_Y}")
    
    
    print("\n 1. CUSTOM IMPLEMENTATION RESULTS:")
    score, aligned1, aligned2, matrix = smith_waterman_local_alignment(
        sequence_X, sequence_Y, 
        match_score=2, mismatch_score=-1, gap_penalty=-1
    )
    
    print(f"Best local alignment score: {score}")
    print(f"Aligned subsequence 1: {aligned1}")
    print(f"Aligned subsequence 2: {aligned2}")
    
    
    print("\nAlignment visualization:")
    print(f"X: {aligned1}")
    print(f"   {'|' * len([i for i, j in zip(aligned1, aligned2) if i == j])}")
    print(f"Y: {aligned2}")
    
    
    
    print("2. BIOPYTHON BUILT-IN ALIGNER RESULTS:")
    best_alignment = align_with_biopython(sequence_X, sequence_Y)
    
    print(f"Alignment score: {best_alignment.score}")
    print(f"Aligned sequences:\n{best_alignment}")
    
    
    
    print("SCORING MATRIX (first few positions):")
    print("     " + "  ".join(sequence_Y[:8]))
    for i in range(min(9, len(sequence_X) + 1)):
        row = sequence_X[i-1] if i > 0 else ' '
        print(f"{row:2} " + " ".join(f"{int(matrix[i][j]):3}" for j in range(min(9, len(sequence_Y) + 1))))

SMITH-WATERMAN LOCAL ALIGNMENT
Sequence X: MAGNIFICENT
Sequence Y: MAGNIFIC


NameError: name 'smith_waterman_local_alignment' is not defined

In [ ]:
def align_with_biopython(seq1, seq2):
    """
    Use Biopython's PairwiseAligner for Needleman-Wunsch alignment
    """
    
    aligner = Align.PairwiseAligner()
    
    
    aligner.mode = 'global'
    
    
    aligner.match_score = 1.0
    aligner.mismatch_score = -1.0
    aligner.open_gap_score = -2.0
    aligner.extend_gap_score = -2.0
    
    
    alignments = aligner.align(seq1, seq2)
    
    
    best_alignment = next(alignments)
    
    return best_alignment



In [ ]:
def calculate_identity(aligned_seq1, aligned_seq2):
    """Calculate sequence identity percentage"""
    matches = sum(1 for a, b in zip(aligned_seq1, aligned_seq2) if a == b and a != '-')
    length = len(aligned_seq1)
    return (matches / length) * 100


if __name__ == "__main__":
    
    sequence_A = "ATCGATCGATCGATCG"
    sequence_B = "ATCGATCG"
    
    print("NEEDLEMAN-WUNSCH GLOBAL ALIGNMENT")
    print(f"Sequence A: {sequence_A}")
    print(f"Sequence B: {sequence_B}")
    print(f"Length A: {len(sequence_A)} bases")
    print(f"Length B: {len(sequence_B)} bases")
    
    
    print("\n1. CUSTOM IMPLEMENTATION RESULTS:")
    score, aligned1, aligned2, matrix = needleman_wunsch_global_alignment(
        sequence_A, sequence_B, 
        match_score=1, mismatch_score=-1, gap_penalty=-2
    )
    
    print(f"Optimal alignment score: {score}")
    print(f"Aligned sequence A: {aligned1}")
    print(f"Aligned sequence B: {aligned2}")
    
    
    identity = calculate_identity(aligned1, aligned2)
    print(f"Sequence identity: {identity:.2f}%")
    
    
    print("\nAlignment visualization:")
    print(f"A: {aligned1}")
    
    
    match_line = ''
    for a, b in zip(aligned1, aligned2):
        if a == b and a != '-':
            match_line += '|'
        elif a != b and a != '-' and b != '-':
            match_line += '.'
        else:
            match_line += ' '
    
    print(f"   {match_line}")
    print(f"B: {aligned2}")
    
    
    print("2. BIOPYTHON BUILT-IN ALIGNER RESULTS:")
    best_alignment = align_with_biopython(sequence_A, sequence_B)
    
    print(f"Alignment score: {best_alignment.score}")
    print(f"Aligned sequences:")
    
    
    alignment_str = str(best_alignment)
    lines = alignment_str.split('\n')
    for i, line in enumerate(lines):
        if i < 3:  
            print(f"  {line}")
    
    
    print("SCORING MATRIX (first 9 positions):")
    print("     " + "  ".join(sequence_B[:8]))
    for i in range(min(9, len(sequence_A) + 1)):
        row = sequence_A[i-1] if i > 0 else ' '
        print(f"{row:2} " + " ".join(f"{int(matrix[i][j]):3}" for j in range(min(9, len(sequence_B) + 1))))




In [ ]:
def analyze_alignment(seq_a, seq_b):
    """Provide detailed analysis of the alignment"""
    print("ALIGNMENT ANALYSIS")
    
    aligner = Align.PairwiseAligner()
    aligner.mode = 'global'
    aligner.match_score = 1.0
    aligner.mismatch_score = -1.0
    aligner.open_gap_score = -1.0
    aligner.extend_gap_score = -1.0
    
    alignments = aligner.align(seq_a, seq_b)
    best_alignment = next(alignments)
    
    aligned_a, aligned_b = str(best_alignment).split('\n')[0], str(best_alignment).split('\n')[2]
    
    
    matches = sum(1 for a, b in zip(aligned_a, aligned_b) if a == b and a != '-')
    mismatches = sum(1 for a, b in zip(aligned_a, aligned_b) if a != b and a != '-' and b != '-')
    gaps = aligned_a.count('-') + aligned_b.count('-')
    
    print(f"Total alignment length: {len(aligned_a)}")
    print(f"Matches: {matches}")
    print(f"Mismatches: {mismatches}")
    print(f"Gaps: {gaps}")
    print(f"Identity: {(matches/len(aligned_a))*100:.2f}%")
    
    return matches, mismatches, gaps


analyze_alignment(sequence_A, sequence_B)